<a href="https://colab.research.google.com/github/BritCrit/media_ocean_IO_checks/blob/main/media_ocean_login_inbox_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Obtain Latest Order List by Region 
import getpass
print('Note username & password for Media Ocean will be deleted on close')
username = input("Username: ")
password = getpass.getpass('Password:')

!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
from google.colab import files
import json
import pandas as pd
import time
from selenium import webdriver
import requests
import sys


def driver_setup():
  sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  return webdriver.Chrome('chromedriver',options=chrome_options)

inbox_url = "https://sellers.mediaocean.com/VendorPortal/routes/vendor/35-2-RGWH-1/inbox/agencies?agencyPublicIds=&numDaysHistory=365&sellerResponseRequired=false&type=all&rfpChecksum=null&orderChecksum=null&converge=true"

def media_ocean_login(driver):
    driver.get("https://sellers.mediaocean.com/")
    time.sleep(5)

    driver.find_element("xpath", '//*[@id="username-input"]').send_keys(username)
    driver.find_element("xpath", '//*[@id="password-input"]').send_keys(password)
    time.sleep(5)
    login = driver.find_element("xpath", '//*[@id="buttonSignin"]')
    login.click()
    time.sleep(5)

def flatten_normalize_json(filename, record):
    with open(filename, 'r') as f:
        data = json.loads(f.read())
        return pd.json_normalize(data, record_path=[record])

def obtain_cookies():
  return driver.get_cookies()

def obtain_inbox_json():
  inbox = s.get(inbox_url)
  return inbox.json()

def region_finder(string):
    kw_region = ['SCR', 'SER', 'WER', 'NCR', 'NER']
    term_return = 'None'
    for term in kw_region:
        if term in string:
            term_return = term
    return term_return

def tactic_finder(string):
    string = string.lower()
    tactics = ['long form', 'short form', 'display', 'search his', 'search', 'tv sync', 'social', 'youtube', 'sponsorship']
    term_return = 'None'
    for term in tactics:
        if term in string:
            term_return = term
    return term_return

# setup driver 
driver = driver_setup()

# login to media ocean
media_ocean_login(driver)

# obtain cookies
cookies = obtain_cookies()

# establish session
s = requests.Session()

# extract cookies
for cookie in cookies:
    s.cookies.set(cookie['name'], cookie['value'])

# obtain inbox json data
inbox_json_data = obtain_inbox_json()

# save json to local file
with open('data.json', 'w') as f:
    json.dump(inbox_json_data, f)

# normalize json data into dataframe
download_inbox = flatten_normalize_json('data.json', ['inboxItems'])

# Filter dataframe to key columns
download_inbox = download_inbox[['sentOn','dueDate','isLastAcceptedOrder','clientName','campaignName','publicId', 'majorVersion', 'endDate', 'endDate', 'status.description', 'cost.amount', 'author', 'buyer.firstName', 'buyer.lastName']]

# convert int64 into time stamp 
download_inbox['sentOn'] = pd.to_datetime(download_inbox['sentOn'], unit='ms')

# filter orders to O based orders
download_inbox[download_inbox['publicId'].astype(str).str.startswith('O')]

# extract region from inbox
download_inbox['region'] = download_inbox['campaignName'].apply(region_finder)

# extract tactic details
download_inbox['tactic'] = download_inbox['campaignName'].apply(tactic_finder)

# set time string for file 
timestr = time.strftime("%m-%d-%Y")
csvfilename = f'{timestr}_media_orders_inbox.csv'

# save csv locally 
download_inbox.to_csv(csvfilename, index=False)

# download csv 
files.download(csvfilename) 
